In [1]:
import os

os.chdir("..")

In [2]:
from neuralforecast.models import NBEATS
from neuralforecast import NeuralForecast
from utils import load_dataset, train_test_split
from models.NBeatsMoe import NBeatsMoe, NBEATSMoEBlock
import torch.nn as nn
import pandas as pd

# datasets list
datasets = [
    {
        "name": "gluonts_tourism_monthly"
    },
    {
        "name": "m3",
        "directory": "C:\\Users\\ricar\\mixture_of_experts_time_series\\data\\m3\\",
        "group": "Monthly",
        "freq": "M",
    } 
]



c:\Users\ricar\mixture_of_experts_time_series\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2025-04-17 01:31:34,308	INFO util.py:154 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.
2025-04-17 01:31:34,466	INFO util.py:154 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.
c:\Users\ricar\mixture_of_experts_time_series\venv\Lib\site-packages\gluonts\json.py:102: UserWarning: Using `json`-module for json-handling. Consider installing one of `orjson`, `ujson` to speed up serialization and deserialization.
  warnings.warn(


In [ ]:
Y_ALL_tourism, horizon, n_lags, group, _ = load_dataset("gluonts_tourism_monthly", datasets[0])
Y_ALL_m3 = load_dataset("m3", datasets[1])

INFO:root:using dataset already processed in path C:\Users\ricar\.gluonts\datasets\tourism_monthly.


Loading tourism_monthly dataset...
Loading m3_monthly dataset...


In [ ]:
print(n_lags)
Y_train_df_tourism, Y_test_df_tourism = train_test_split(Y_ALL_tourism, horizon)
Y_train_df_m3, Y_test_df_m3 = train_test_split(Y_ALL_m3, horizon)

24


In [ ]:
models_tourism = NBEATS(h=horizon, input_size=n_lags)
models_m3 = NBEATS(h=horizon, input_size=n_lags)

Seed set to 1
Seed set to 1


In [ ]:
fcst_tourism = NeuralForecast(models=[models_tourism], freq="M")
fcst_tourism.fit(df=Y_train_df_tourism, static_df=None, val_size=horizon)

fcst_m3 = NeuralForecast(models=[models_m3], freq="M")
fcst_m3.fit(df=Y_train_df_m3, static_df=None, val_size=horizon)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name         | Type          | Params | Mode 
-------------------------------------------------------
0 | loss         | MAE           | 0      | train
1 | padder_train | ConstantPad1d | 0      | train
2 | scaler       | TemporalNorm  | 0      | train
3 | blocks       | ModuleList    | 2.5 M  | train
-------------------------------------------------------
2.5 M     Trainable params
1.6 K     Non-trainable params
2.5 M     Total params
9.853     Total estimated model params size (MB)
31        Modules in train mode
0         Modules in eval mode


Epoch 83:  33%|███▎      | 4/12 [00:01<00:02,  3.61it/s, v_num=68, train_loss_step=1.15e+3, train_loss_epoch=893.0, valid_loss=2.65e+3]   

`Trainer.fit` stopped: `max_steps=1000` reached.


Epoch 83:  33%|███▎      | 4/12 [00:01<00:02,  3.61it/s, v_num=68, train_loss_step=1.15e+3, train_loss_epoch=893.0, valid_loss=2.65e+3]


GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name         | Type          | Params | Mode 
-------------------------------------------------------
0 | loss         | MAE           | 0      | train
1 | padder_train | ConstantPad1d | 0      | train
2 | scaler       | TemporalNorm  | 0      | train
3 | blocks       | ModuleList    | 2.5 M  | train
-------------------------------------------------------
2.5 M     Trainable params
1.6 K     Non-trainable params
2.5 M     Total params
9.853     Total estimated model params size (MB)
31        Modules in train mode
0         Modules in eval mode


Epoch 22:  22%|██▏       | 10/45 [00:02<00:07,  4.74it/s, v_num=69, train_loss_step=488.0, train_loss_epoch=579.0, valid_loss=665.0]

`Trainer.fit` stopped: `max_steps=1000` reached.


Epoch 22:  22%|██▏       | 10/45 [00:02<00:07,  4.74it/s, v_num=69, train_loss_step=488.0, train_loss_epoch=579.0, valid_loss=665.0]


Now lets add the 2 datasets together ...

In [ ]:
blcs_tourism = fcst_tourism.models[0].blocks
blcs_m3 = fcst_m3.models[0].blocks

... and the 2 blocks

In [ ]:
added_layers = nn.ModuleList()
for i in zip(blcs_tourism, blcs_m3):
    blc1, blc2 = i
    n_theta = blc1.layers[-1].out_features
    added_layers.append(NBEATSMoEBlock(
        input_size=n_lags,
        top_k=1,
        nr_experts=2,
        n_theta=n_theta,
        mlp_units=[[16,16], [16,16], [16,16]], ## just moked
        basis=blc1.basis,
        activation="ReLU",
        dropout_prob=blc1.dropout_prob,
        pre_experts=nn.ModuleList([blc1.layers, blc2.layers]),
    ))
added_layers

ModuleList(
  (0): NBEATSMoEBlock(
    (experts): ModuleList(
      (0-1): 2 x Sequential(
        (0): Linear(in_features=24, out_features=512, bias=True)
        (1): Linear(in_features=512, out_features=512, bias=True)
        (2): ReLU()
        (3): Linear(in_features=512, out_features=512, bias=True)
        (4): ReLU()
        (5): Linear(in_features=512, out_features=512, bias=True)
        (6): ReLU()
        (7): Linear(in_features=512, out_features=42, bias=True)
      )
    )
    (gate): Sequential(
      (0): LayerNorm((24,), eps=1e-05, elementwise_affine=True)
      (1): Linear(in_features=24, out_features=2, bias=True)
    )
    (softmax): Softmax(dim=1)
    (layers): Sequential(
      (0): Linear(in_features=24, out_features=16, bias=True)
      (1): Linear(in_features=16, out_features=16, bias=True)
      (2): ReLU()
      (3): Linear(in_features=16, out_features=16, bias=True)
      (4): ReLU()
      (5): Linear(in_features=16, out_features=16, bias=True)
      (6): R

In [ ]:
moe = NBeatsMoe(
    pre_blocks=added_layers,
    nr_experts=2,
    top_k=1,
    input_size=n_lags,
    h=horizon,
    # learning_rate=1e-5,
    # max_steps=1000,
)

Seed set to 1


Let's train ...

In [ ]:
## add both y_all
Y_ALL = Y_ALL_tourism.copy()
Y_ALL = pd.concat([Y_ALL, Y_ALL_m3], ignore_index=True)

# split train and test
Y_train_df, Y_test_df = train_test_split(Y_ALL, horizon)

In [ ]:
fcts_tl = NeuralForecast(models=[moe], freq="M")
fcts_tl.fit(df=Y_train_df, static_df=None, val_size=horizon)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name         | Type          | Params | Mode 
-------------------------------------------------------
0 | loss         | MAE           | 0      | train
1 | padder_train | ConstantPad1d | 0      | train
2 | scaler       | TemporalNorm  | 0      | train
3 | blocks       | ModuleList    | 4.9 M  | train
-------------------------------------------------------
4.9 M     Trainable params
1.6 K     Non-trainable params
4.9 M     Total params
19.724    Total estimated model params size (MB)
97        Modules in train mode
0         Modules in eval mode


Sanity Checking DataLoader 0:   0%|          | 0/2 [00:00<?, ?it/s]expert_output: torch.Size([32, 42])
expert_weight: torch.Size([32])
expert_output: torch.Size([32, 42])
expert_weight: torch.Size([32])
expert_output: torch.Size([32, 6])
expert_weight: torch.Size([32])
expert_output: torch.Size([32, 6])
expert_weight: torch.Size([32])
expert_output: torch.Size([32, 68])
expert_weight: torch.Size([32])
expert_output: torch.Size([32, 68])
expert_weight: torch.Size([32])
Sanity Checking DataLoader 0:  50%|█████     | 1/2 [00:00<00:00, 45.49it/s]expert_output: torch.Size([32, 42])
expert_weight: torch.Size([32])
expert_output: torch.Size([32, 42])
expert_weight: torch.Size([32])
expert_output: torch.Size([32, 6])
expert_weight: torch.Size([32])
expert_output: torch.Size([32, 6])
expert_weight: torch.Size([32])
expert_output: torch.Size([32, 68])
expert_weight: torch.Size([32])
expert_output: torch.Size([32, 68])
expert_weight: torch.Size([32])
Epoch 0:   0%|          | 0/57 [00:00<?, ?it/s


Detected KeyboardInterrupt, attempting graceful shutdown ...


NameError: name 'exit' is not defined

In [ ]:
# lets predict ...
predictions = fcts_tl.predict(futr_df=Y_test_df)
predictions_tourism = fcst_tourism.predict(futr_df=Y_test_df_tourism)
predictions_m3 = fcst_m3.predict(futr_df=Y_test_df_m3)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Predicting DataLoader 0: 100%|██████████| 57/57 [00:01<00:00, 54.61it/s]


GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Predicting DataLoader 0: 100%|██████████| 12/12 [00:00<00:00, 77.90it/s]


GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Predicting DataLoader 0: 100%|██████████| 45/45 [00:00<00:00, 87.92it/s]


In [ ]:
from neuralforecast.losses.numpy import smape

# Calculate SMAPE for each prediction

# For predictions from fcts_tl
def calculate_smape(Y_test, predictions, model_name):
    y_true = Y_test['y'].values
    y_hat = predictions[model_name].values

    n_series = Y_test['unique_id'].nunique()
    y_true = y_true.reshape(n_series, -1)
    y_hat = y_hat.reshape(n_series, -1)

    smape_value = smape(y_true, y_hat)
    return smape_value

# Calculate SMAPE for fcts_tl
smape_tl = calculate_smape(Y_test_df, predictions, moe.__class__.__name__)
smape_tourism = calculate_smape(Y_test_df_tourism, predictions_tourism, "NBEATS")
smape_m3 = calculate_smape(Y_test_df_m3, predictions_m3, "NBEATS")

# Print SMAPE values
print(f"SMAPE for fcts_tl: {smape_tl}")
print(f"SMAPE for fcst_tourism: {smape_tourism}")
print(f"SMAPE for fcst_m3: {smape_m3}")

SMAPE for fcts_tl: 0.2238958588898837
SMAPE for fcst_tourism: 0.24402993926027292
SMAPE for fcst_m3: 0.13870548088695284


Test the moe on both datasets independently ....

In [ ]:
predictions_tl_tourism = predictions[~predictions['unique_id'].str.match(r'^M\d+')]
predictions_tl_m3 = predictions[predictions['unique_id'].str.match(r'^M\d+')]

# Calculate SMAPE for fcts_tl
smape_tl_tourism = calculate_smape(Y_test_df_tourism, predictions_tl_tourism, moe.__class__.__name__)
smape_tl_m3 = calculate_smape(Y_test_df_m3, predictions_tl_m3, moe.__class__.__name__)

# Print SMAPE values
print(f"SMAPE for fcts_tl_tourism: {smape_tl_tourism}")
print(f"SMAPE for fcts_tl_m3: {smape_tl_m3}")

SMAPE for fcts_tl_tourism: 0.4129302430484678
SMAPE for fcts_tl_m3: 0.1754458696727676


predict on other dataset ... 
